In [6]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from openai_services import ai_flow

In [ ]:
openai_api_kay = ""

In [10]:
from bert_score import score

def avaliar_componentes_com_bertscore(texto_gerado, componentes_esperados, threshold=0.6):
    """
    Verifica se os componentes esperados estão presentes no texto gerado,
    com base na similaridade semântica (BERTScore F1).

    Args:
        texto_gerado (str): Texto gerado pelo modelo LLM.
        componentes_esperados (List[str]): Lista dos nomes dos componentes que devem ser identificados.
        threshold (float): Valor mínimo do F1-score para considerar um componente detectado (default: 0.6)

    Returns:
        List[Tuple[str, float]]: Lista de componentes detectados com seus respectivos F1 scores.
    """
    detectados = []

    for componente in componentes_esperados:
        # Frase de referência artificial para gerar contexto
        referencia = [f"This mentions {componente}"]
        candidato = [texto_gerado]

        _, _, F1 = score(candidato, referencia, lang="en", verbose=False)

        if F1[0].item() >= threshold:
            detectados.append((componente, round(F1[0].item(), 3)))

    return detectados


In [12]:
chat = ai_flow.Chat(openai_api_key=openai_api_kay, model="o4-mini-2025-04-16")

In [27]:
import json
import logging

def extrair_componentes_de_imagem(img_path):
    """
    Recebe a imagem e o handler de leitura (ex: chat.read_architecture),
    retorna a lista de componentes identificados.

    Args:
        arquitetura: imagem ou caminho da imagem da arquitetura
        chat_handler: função que processa a imagem e retorna JSON (ex: chat.read_architecture)

    Returns:
        List[str]: lista de componentes identificados
    """
    try:
        with open(img_path, "rb") as img_file:
            response = chat.read_architecture(img_file)

        # Limpeza básica da resposta textuals
        response = response.replace('```', '').replace('json', '').strip()

        resultado = json.loads(response)

        return resultado.get("componentes_identificados", [])

    except json.JSONDecodeError as e:
        logging.error(f"Erro ao decodificar JSON: {str(e)}\nResposta: {response}")
        return []


In [41]:
import os
import pandas as pd

def pipeline_avaliacao(
    pasta_imagens,
    listas_esperadas,
    threshold=0.6,
    extensao=".png"
):
    resultados = []

    arquivos = sorted([f for f in os.listdir(pasta_imagens) if f.endswith(extensao)])

    for idx, nome_arquivo in enumerate(arquivos):
        caminho_imagem = os.path.join(pasta_imagens, nome_arquivo)
        lista_esperada = listas_esperadas[idx]

        print(f"🔍 Processando {nome_arquivo}...")

        # Extrai componentes detectados
        detectados = extrair_componentes_de_imagem(caminho_imagem)

        # Junta os componentes detectados em string para BERTScore
        texto_gerado = ", ".join(detectados)

        # Avalia com BERTScore
        avaliados = avaliar_componentes_com_bertscore(texto_gerado, lista_esperada, threshold=threshold)

        # Extrai só os nomes dos que foram considerados corretos
        corretos = [comp for comp, _ in avaliados]

        # F1 médio dos componentes corretos
        f1_medio = round(sum(f1 for _, f1 in avaliados) / len(avaliados), 3) if avaliados else 0.0

        resultados.append({
            "imagem": nome_arquivo,
            "componentes_esperados": lista_esperada,
            "componentes_detectados": detectados,
            "componentes_corretos": corretos,
            "f1_medio": f1_medio
        })

    return resultados


In [45]:
# Caminho da pasta com imagens
pasta = "bert-dataset/"

import warnings
warnings.filterwarnings("ignore")

# Listas esperadas, na mesma ordem dos arquivos na pasta
listas_esperadas = [
    ['Usuarios', 'AWS WAF', 'AWS CloudFront', 'AWS S3', 'AWS API Gateway', 'AWS lambda', 'AWS SES', 'AWS Dynamo DB'],
    ['Clinical documents', 'Direct Connect', 'Amazon S3', 'Amazon Textract', 'Amazon OpenSearch', 'AWS Lambda', 'Amazon EC2', 'Amazon EKS', 'Amazon MQ', 'Amazon Sagemaker', 'Amazon Bedrock', 'Amazon RDS'],
    ['Migration automation Server', 'Amazon Cloudfront', 'Amazon API Gateway', 'AWS Secrets Manager', 'AWS S3', 'AWS Lambda', 'AWS Systems Manager', 'Amazon Cognito', 'Amazon CloudEndure', 'Amazon Dynamo DB', 'Amazon EC2', 'Amazon MGN', 'Amazon Quicksight'],
    ['Users', 'Client Devices ', 'Azure Function', 'Azure Events Hubs', 'Azure Event Grid', 'Azure Blob Storage', 'Applications/Service', 'Azure data Explorer', 'Metrics Advisor', 'Grafana'],
    ['HTTP Traffic', 'Ingestion Service', 'Azure service Bus', 'Managed Identities', 'Azure key vault', 'Azure application Insights', 'Azure Monitor', 'Azure log analytics', 'Workflow service', 'Package service', 'Container app', 'Azure CosmosDB', 'Azure Cosmos for MongoDB API', 'Azure Cache for Redis'],
    ['Customer', 'Browser', 'Azure CDN', 'Blob Storage', 'Virtual Network', 'Web App Content', 'Redis Cache', 'SQL Database', 'API App', 'Application Gateway', 'Azure active Directory', 'Application Insights', 'Emproyee']
]

# Rodar o pipeline
resultado_final = pipeline_avaliacao(pasta, listas_esperadas)

# Transformar em DataFrame
df_resultados = pd.DataFrame(resultado_final)

# Ver no console
print(df_resultados)

# Opcional: salvar em CSV
df_resultados.to_csv("avaliacao_arquiteturas.csv", index=False)


INFO:root:Enviando mensagem para análise de arquitetura.


🔍 Processando aws_1.png...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Análise de arquitetura concluída.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not

🔍 Processando aws_2.png...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Análise de arquitetura concluída.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not

...

🔍 Processando azure_2.png...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Análise de arquitetura concluída.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not

🔍 Processando azure_3.png...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Análise de arquitetura concluída.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not

        imagem                              componentes_esperados  \
0    aws_1.png  [Usuarios, AWS WAF, AWS CloudFront, AWS S3, AW...   
1    aws_2.png  [Clinical documents, Direct Connect, Amazon S3...   
2    aws_3.png  [Migration automation Server, Amazon Cloudfron...   
3  azure_1.png  [Users, Client Devices , Azure Function, Azure...   
4  azure_2.png  [HTTP Traffic, Ingestion Service, Azure servic...   
5  azure_3.png  [Customer, Browser, Azure CDN, Blob Storage, V...   

                              componentes_detectados  \
0  [AWS - WAF, AWS - CloudFront, AWS - API Gatewa...   
1  [On-Premises - Clinical Documents, AWS - Direc...   
2  [On-Premises Migration Automation Server, Amaz...   
3  [Azure Function (Collect device telemetry), Az...   
4  [HTTP Ingress (entrada de tráfego HTTP), Azure...   
5  [Azure Blob Storage, Azure CDN, Azure Applicat...   

                                componentes_corretos  f1_medio  
0  [Usuarios, AWS WAF, AWS CloudFront, AWS S3, AW...     0

In [46]:
df_resultados

,imagem,componentes_esperados,componentes_detectados,componentes_corretos,f1_medio
0,aws_1.png,"[Usuarios, AWS WAF, AWS CloudFront, AWS S3, AW...","[AWS - WAF, AWS - CloudFront, AWS - API Gatewa...","[Usuarios, AWS WAF, AWS CloudFront, AWS S3, AW...",0.811
1,aws_2.png,"[Clinical documents, Direct Connect, Amazon S3...","[On-Premises - Clinical Documents, AWS - Direc...","[Clinical documents, Direct Connect, Amazon S3...",0.806
2,aws_3.png,"[Migration automation Server, Amazon Cloudfron...","[On-Premises Migration Automation Server, Amaz...","[Migration automation Server, Amazon Cloudfron...",0.813
3,azure_1.png,"[Users, Client Devices , Azure Function, Azure...","[Azure Function (Collect device telemetry), Az...","[Users, Client Devices , Azure Function, Azure...",0.830
4,azure_2.png,"[HTTP Traffic, Ingestion Service, Azure servic...","[HTTP Ingress (entrada de tráfego HTTP), Azure...","[HTTP Traffic, Ingestion Service, Azure servic...",0.812
5,azure_3.png,"[Customer, Browser, Azure CDN, Blob Storage, V...","[Azure Blob Storage, Azure CDN, Azure Applicat...","[Customer, Browser, Azure CDN, Blob Storage, V...",0.800


## 📊 Análise dos Resultados

A avaliação foi conduzida sobre seis diagramas distintos — três representando arquiteturas baseadas em serviços da AWS e três em serviços da Azure. Para cada imagem, foi fornecida uma lista de componentes esperados, construída manualmente com base no conteúdo visual presente nos diagramas.

### Desempenho geral

Os resultados mostram que o modelo alcançou **F1 Scores médios entre 0.80 e 0.83** em todos os casos, com destaque para:

- **`azure_1.png`** com **F1 de 0.83**, demonstrando ótima correspondência mesmo com nomes genéricos como `"Users"` e `"Client Devices"`.
- **Consistência entre todas as imagens**, com uma variação de F1 muito pequena (apenas 0.03 entre o maior e o menor valor).

### Observações importantes

- O modelo demonstrou forte **capacidade de generalização semântica**, conseguindo mapear corretamente variações como:
  - `"AWS - Lambda (Gerador de Permissões)"` → `"AWS Lambda"`
  - `"Amazon S3 (Frontend Code)"` → `"Amazon S3"`
  - `"Azure Cosmos DB for MongoDB API"` → `"Azure Cosmos for MongoDB API"`

- Apesar das variações na forma textual, **todos os componentes esperados foram corretamente identificados** em todos os casos, segundo o critério de similaridade contextual fornecido pelo BERTScore.

- **Componentes compostos** e **descrições longas** não comprometeram a avaliação: o modelo obteve boas pontuações mesmo em casos como `"AWS Lambda (Login Functions)"` e `"Azure Web App - Content Website"`, o que reforça sua capacidade de compreender a função semântica dos termos.

### Ponto de destaque

- A arquitetura **`aws_3.png`**, com 13 componentes esperados, foi corretamente reconhecida com F1 de **0.813**, evidenciando a escalabilidade da abordagem mesmo em diagramas mais densos e complexos.

### Considerações finais

Apesar do uso exclusivo do BERTScore — que é mais sensível a contexto do que métodos com embeddings fixos —, o modelo foi capaz de capturar corretamente os conceitos principais das arquiteturas analisadas. A abordagem se mostrou adequada para casos em que se deseja avaliar não apenas igualdade textual, mas também compreensão conceitual dos elementos presentes em uma imagem.


## 📌 Conclusão

Esta avaliação analisou a capacidade de um modelo de linguagem em identificar componentes arquiteturais a partir de diagramas em imagens, utilizando como métrica o **BERTScore**. Foram comparadas listas de componentes esperados (extraídas manualmente das imagens) com os componentes retornados pelo modelo, considerando similaridade semântica e não apenas igualdade literal.

O **BERTScore** calcula a similaridade entre pares de palavras com base em embeddings contextuais do modelo BERT, permitindo capturar correspondências mesmo com variações na forma de escrita, nomes compostos ou descrições estendidas. Isso se mostrou particularmente útil para componentes como `"AWS Lambda (Validador de Permissões)"` e `"AWS Lambda"`, que receberam alta pontuação mesmo com diferenças textuais.

Os resultados demonstraram alta precisão semântica: em todas as imagens analisadas, o F1 médio permaneceu **acima de 0,80**, indicando que o modelo foi capaz de identificar corretamente a grande maioria dos elementos esperados nas arquiteturas.

Comparado a métodos que usam apenas embeddings fixos com similaridade de cosseno (como `sentence-transformers`), o uso do BERTScore tende a ser mais rigoroso e contextual, oferecendo avaliações mais precisas em casos de ambiguidade ou descrições mais longas — embora com maior custo computacional.
